# Assignment: Probability, Bayesian Probability, and Gradient Descent
_Group 6_

### Group Members
- David Akintayo: Probability Distributions  
- Cynthia Mutie: Bayesian Probability  
- Sougnabe Payang: Manual Gradient Descent  
- Elvis Kayonga: Linear Regression with SciPy 

## Part 0 — Data Setup
Load and explore the dataset to be used across all sections.


In [2]:
import pandas as pd

# Name the columns so we understand the data
col_names = ["sepal_length", "sepal_width", "petal_length", "petal_width", "species"]

# Download the data from the internet and load it
df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data",
    header=None, names=col_names
)

# Show the first 5 rows
df.head()


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


## Part 1 — Probability Distributions (David Akintayo)

Implement and compare key **probability distributions** using the Iris dataset.  
Tasks:
- Compute and visualize probability distributions (e.g., Normal, Uniform, Exponential).  
- Plot histograms and fit curves for each feature.  
- Analyze which distribution best fits each variable.


## Part 2 — Bayesian Probability (Cynthia Mutie)
Dataset: IMDb Movie Reviews (50k) — file in repo: `data/IMDB Dataset.csv`

Objective: For chosen keywords compute:
- Prior: P(Positive)
- Likelihood: P(keyword | Positive)
- Marginal: P(keyword)
- Posterior: P(Positive | keyword)

We will compute P(Positive | keyword) using plain Python + pandas only.


## Part 3 — Manual Gradient Descent (Sougnabe Payang)

Manually calculate the **gradient descent steps** for a simple cost function:  
\[
J(m, b) = \frac{1}{n}\sum_{i=1}^n (y_i - (mx_i + b))^2
\]
Tasks:
- Derive partial derivatives of \(J\) with respect to \(m\) and \(b\).  
- Implement manual gradient descent using loops.


## Part 4 — Gradient Descent with SciPy (Elvis Kayonga)

Use the **SciPy library** to perform linear regression using optimization tools.  
Compare manual vs. automatic gradient descent results.
